In order to calculate WTD from logger data we do the following:
1. Adjust the measured pressure by negating atmospheric pressure.
2. Convert the measured pressure from mmHg to mmH2O using multiplication by 13.5951. This gives us (f) in the diagram below.
3. Use the equations in the diagram below to convert field measurements to water table depth.


<div>
<img src="logger_diagram.png" width="500"/>
</div>

In [4]:
#IMPORT LIBRARIES
import os
import pandas as pd
import numpy as np
import seaborn as sns

In [5]:
# MAKE A DIRECTORY TO EXPORT PROCESSED WATER TABLE TO
try:
    os.mkdir('EXPORT')
except FileExistsError:
    pass

In [18]:
#DEFINE CONSTANTS

#The following constant converts mmHg to mmH20
HG_TO_H2O = 13.5951 #This is based on the pressure of 1 mm of H2O at 4C 
# This value will vary slightly with temperature but we are ignoring that

#Altitudes
# It's important to know the altitude of each logger and its associated barometer
# Pressure decreases with altitude and this needs to be accounted for
#Altitude and pressure have a non-linear relationship, but for the purposes of our loggers we give an approximate linear estimate
ALT_TO_HG = -0.0875 # Atmospheric pressure decreaess ~ -0.0875 mmHg per m (calculated manually using https://www.sensorsone.com/altitude-pressure-units-conversion/)
# Barometer altitudes in m
# Waiting to get the GPS data for the Forsinard Flows Barometer
BaroAlts = {'FF': 0,
            'Braemore': 294, 
            'Scaraben': 294,
            'Strathy': 294, 
            'Morven': 294,
            'Wag': 294,
            'Mig': 455,
            'MH': 555 
           }
# Logger altitudes in m 
LoggerAlts = {'FF':{'DAMA':187,'DAMB':187,'DAMC':187,'NND':184,'NNE':183,'NNF':185,'RESG':176,'RESH':177,'RESI':174},
     'Mig':{'NN1':458,'RES1':456,'RES2':455,'DAM1':459,'DAM2':455,'DAM3':453},
     'MH':{'NN1':556,'NN2':554,'RES1':561,'RES2':565,'DAM1':550,'DAM2':550},
     'Braemore':{'NN1':296,'NN2':296},
     'Scaraben':{'RES1':233,'RES2':235},
     'Strathy':{'RES1':143,'RES2':140},
     'Morven':{'DAM1':279,'DAM2':265},
     'Wag':{'RES1':268,'RES2':268}}

#Dipwell lip to logger mount (mm)
A = 20
#Dipwell lip to peat surface (mm)
B = {'FF':{'A':63,'B':59,'C':80,'D':64,'E':75,'F':62,'G':77,'H':55,'I':180},
     'Mig':50,
     'MH':{'RES':100,'NN':100,'DAM':50},
     'Braemore':{'NN1':50,'NN2':38},
     'Scaraben':{'RES1':43,'RES2':39},
     'Strathy':{'RES1':44,'RES2':40},
     'Morven':{'DAM1':51,'DAM2':48},
     'Wag':{'RES1':51,'RES2':64}}
#Length of logger wire to bottom of sensor (mm)
#Add 130 to Migneint to compensate for mismeasurement of wire when installing
C = {'FF':{'A':980,'B':970,'C':975,'D':980,'E':980,'F':980,'G':850,'H':985,'I':835},
     'Mig':{'NN1':880+130,'RES1':880+130,'RES2':880+130,'DAM1':880+130,'DAM2':880+130,'DAM3':880+130},
     'MH':880,
     'Braemore':{'NN1':975,'NN2':965},
     'Scaraben':{'RES1':970,'RES2':975},
     'Strathy':{'RES1':985,'RES2':975},
     'Morven':{'DAM1':840,'DAM2':980},
     'Wag':{'RES1':970,'RES2':980}}

#Define the column titles in the logger dataframe
columnTitles = ['ID','Site','Region','Datetime','Date','Hour','Day','Week','Month','Quarter','Year','Temperature(Logger)','mmHg(Logger)','mmH2O','A','B','C','E','WTD(mm)']
#Define the order of the column titles in the final dataframe 
columnOrder = ['ID','Site','Region','Datetime','Date','Hour','Day','Week','Month','Quarter','Year','Temperature(Logger)','Temperature(Baro)','mmHg(Logger)','mmHg(Baro)','mmHg(Baro Adj)','mmH2O','A','B','C','E','WTD(mm)']

#Define the names of the langwell sites
sitesLW = {'Braemore','Strathy','Scaraben','Morven','Wag'}

In [12]:
#This is a dictionary that I can store each processed dataset in 
analysisDict = {}

In [13]:
#Read the barometer data 
MH_Baro = pd.read_csv('CSV//MH_BARO_1_SN918186_2023-02-24_13-03-16-660.csv', skip_blank_lines = True, engine = 'python')
MH_Baro['Date and Time'] = pd.DatetimeIndex(MH_Baro['Date and Time'],dayfirst=True).round("H")
MH_Baro.columns = ['Datetime','Seconds','mmHg(Baro)', 'Temperature(Baro)']
Mig_Baro = pd.read_csv('CSV//Mig_Baro_1_925338_2023-02-23_12-51-52-347.csv', skip_blank_lines = True, engine = 'python')
Mig_Baro['Date and Time'] = pd.DatetimeIndex(Mig_Baro['Date and Time'],dayfirst=True).round("H")
Mig_Baro.columns = ['Datetime','Seconds','mmHg(Baro)', 'Temperature(Baro)']
FF_Baro = pd.read_csv('CSV//FF_VuSitu_2020-02-18_12-00-00_REFAP_Log_Air reference logger.csv', skip_blank_lines = True, engine = 'python', skiprows = 27)
FF_Baro.columns = ['Datetime','mmHg(Baro)', 'Temperature(Baro)']
FF_Baro = FF_Baro.iloc[:-3,:] #remove empty rows
FF_Baro['Datetime'] = pd.DatetimeIndex(FF_Baro['Datetime'],dayfirst=True).round("H")

In [14]:
# The barometer data for Braemore baro is formatted slightly differently 
Braemore_Baro = pd.read_csv('CSV//Braemore_baro_2_2023-08-04_18-57-45-697.csv', skip_blank_lines = True, engine = 'python', skiprows = 63)
Braemore_Baro = Braemore_Baro.iloc[:,:4] #remove empty rows
Braemore_Baro.columns = ['Datetime','Seconds','mmHg(Baro)', 'Temperature(Baro)']
Braemore_Baro = Braemore_Baro.iloc[:-3,:] #remove empty rows
Braemore_Baro['Datetime'] = pd.DatetimeIndex(Braemore_Baro['Datetime'],dayfirst=True).round("H")

In [19]:
#Process the barometer data
for f in os.listdir('CSV'):
    #only parse .csv files
    if f.endswith('.csv'):
        #ignore baro data
        if 'Baro' in f or 'BARO'in f or 'REFAP' in f or 'baro' in f:
            continue 
        # Set up dataframe in analysisDict
        analysisDict[f] = pd.DataFrame(columns=columnTitles)
        # Handle Different site data 
        site = f.split('_')[0]
        region = f.split('_')[1]
        pos = f.split('_')[2]
        uqid = site + region + pos
        # read in dataset
        # forsinard flows (crocach) is in a different format so needs to be read separately 
        if site == 'FF':
            #skip to june 2021 (to speed the processing up), in order to include the whole dataset set skiprows to 27 
            df = pd.read_csv('CSV//'+f, skip_blank_lines = True, engine = 'python', skiprows = 18000) 
            #Give the FF columns the same headers as other files
            df.columns = ['Date and Time', 'Pressure (mmHg)', 'Temperature (C)','Depth to Water (mm)'] 
            #remove na values
            idx = df.loc[pd.isna(df["Pressure (mmHg)"]), :].index
            df = df.drop(idx)
        # the langwell sites (strathy, braemore, scaraben, morven and wag) have a separate format so also need to be read separately (skiprows = 80)
        elif site in sitesLW:
            df = pd.read_csv('CSV//'+f, skip_blank_lines = True, engine = 'python', skiprows = 80)
            df = df.iloc[:,:5]
            df.columns = ['Date and Time', 'Seconds','Pressure (mmHg)', 'Temperature (C)','Depth to Water (mm)'] 
        # moor house and migneint have the same format
        else:
            df = pd.read_csv('CSV//'+f, skip_blank_lines = True, engine = 'python')
        # Convert datetime to datetime index to speed things up and enable better string parsing
        # Round to nearest hour to match barometer data
        analysisDict[f]['Datetime'] = pd.DatetimeIndex(df['Date and Time'],dayfirst=True).round("H")
        #Deal with different sites and merge barometer data
        analysisDict[f].loc[:,'ID'] = uqid
        analysisDict[f].loc[:,'Site'] = site
        analysisDict[f].loc[:,'Region'] = region
        if site == 'MH':
            analysisDict[f].loc[:,'B'] = B[site][region] 
            analysisDict[f].loc[:,'C'] = C[site]
            analysisDict[f] = analysisDict[f].merge(MH_Baro, left_on='Datetime', right_on='Datetime', how = 'left')     
            analysisDict[f].loc[:,'mmHg(Baro Adj)'] = analysisDict[f].loc[:,'mmHg(Baro)'] + ((LoggerAlts[site][region+pos] - BaroAlts[site]) * ALT_TO_HG)
        elif site == 'Mig':
            analysisDict[f].loc[:,'B'] = B[site]
            analysisDict[f].loc[:,'C'] = C[site][region+pos]
            analysisDict[f] = analysisDict[f].merge(Mig_Baro, left_on='Datetime', right_on='Datetime', how = 'left')  
            analysisDict[f].loc[:,'mmHg(Baro Adj)'] = analysisDict[f].loc[:,'mmHg(Baro)'] + ((LoggerAlts[site][region+pos] - BaroAlts[site]) * ALT_TO_HG)
        elif site == 'FF':
            analysisDict[f].loc[:,'B'] = B[site][pos]
            analysisDict[f].loc[:,'C'] = C[site][pos]
            analysisDict[f] = analysisDict[f].merge(FF_Baro, left_on='Datetime', right_on='Datetime', how = 'left')     
            analysisDict[f].loc[:,'mmHg(Baro Adj)'] = analysisDict[f].loc[:,'mmHg(Baro)'] #update when barometer altitude for FF is known
        elif site in sitesLW:
            analysisDict[f].loc[:,'B'] = B[site][region+pos]
            analysisDict[f].loc[:,'C'] = C[site][region+pos]
            analysisDict[f] = analysisDict[f].merge(Braemore_Baro, left_on='Datetime', right_on='Datetime', how = 'left') 
            analysisDict[f].loc[:,'mmHg(Baro Adj)'] = analysisDict[f].loc[:,'mmHg(Baro)'] + ((LoggerAlts[site][region+pos] - BaroAlts[site]) * ALT_TO_HG)
        # Process the datetime into different time measures
        analysisDict[f]['Date'] = analysisDict[f]['Datetime'].dt.date #extract date
        analysisDict[f]['Hour'] = analysisDict[f]['Datetime'].dt.hour #hour
        analysisDict[f]['Day'] = analysisDict[f]['Datetime'].dt.dayofyear #day of year
        analysisDict[f]['Week'] = analysisDict[f]['Datetime'].dt.isocalendar().week.astype(int) #week
        analysisDict[f]['Month'] = analysisDict[f]['Datetime'].dt.month# month
        analysisDict[f]['Year'] = analysisDict[f]['Datetime'].dt.year #and yearly values
        analysisDict[f]['Quarter'] = analysisDict[f]['Datetime'].dt.quarter #quarterly value
        # Calculate WTD 
        analysisDict[f].loc[:,'Temperature(Logger)'] = df.loc[:,'Temperature (C)'].astype(float)
        analysisDict[f].loc[:,'mmHg(Logger)'] = df.loc[:,'Pressure (mmHg)'].astype(float)
        analysisDict[f].loc[:,'mmH2O'] = (analysisDict[f].loc[:,'mmHg(Logger)'] -  analysisDict[f].loc[:,'mmHg(Baro Adj)']) * HG_TO_H2O
        analysisDict[f].loc[:,'A'] = A
        analysisDict[f].loc[:,'E'] = analysisDict[f].loc[:,'C'] - analysisDict[f].loc[:,'B'] + analysisDict[f].loc[:,'A']
        analysisDict[f].loc[:,'WTD(mm)'] = analysisDict[f].loc[:,'E'] -  analysisDict[f].loc[:,'mmH2O']
        #reorder the columns
        analysisDict[f] = analysisDict[f][columnOrder]
        #write to csv files
        analysisDict[f].to_csv(f'EXPORT//p_{f}', index = False)

Merge all individual datasets into one 

In [20]:
mergeDf = pd.DataFrame(columns=columnOrder)
#Merge the barometer data into a single datafram
for df in analysisDict.values():
    mergeDf = pd.merge(mergeDf,df, how = 'outer')
mergeDf.to_csv('EXPORT//p_mergedData.csv', index = False)

In [21]:
mergeDf

,ID,Site,Region,Datetime,Date,Hour,Day,Week,Month,Quarter,...,Temperature(Baro),mmHg(Logger),mmHg(Baro),mmHg(Baro Adj),mmH2O,A,B,C,E,WTD(mm)
0,BraemoreNN1,Braemore,NN,2022-08-03 22:00:00,2022-08-03,22,215,31,8,3,...,9.127,782.481,730.769,730.594,705.408954,20,50,975,945,239.591046
1,BraemoreNN1,Braemore,NN,2022-08-03 23:00:00,2022-08-03,23,215,31,8,3,...,8.185,783.353,731.143,730.968,712.179313,20,50,975,945,232.820687
2,BraemoreNN1,Braemore,NN,2022-08-04 00:00:00,2022-08-04,0,216,31,8,3,...,8.238,783.973,730.980,730.805,722.824277,20,50,975,945,222.175723
3,BraemoreNN1,Braemore,NN,2022-08-04 01:00:00,2022-08-04,1,216,31,8,3,...,8.343,784.745,731.335,731.160,728.493433,20,50,975,945,216.506567
4,BraemoreNN1,Braemore,NN,2022-08-04 02:00:00,2022-08-04,2,216,31,8,3,...,8.657,785.197,731.381,731.206,734.013044,20,50,975,945,210.986956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254659,WagRES2,Wag,RES,2023-08-04 08:00:00,2023-08-04,8,216,31,8,3,...,8.343,795.803,736.306,738.581,777.938812,20,64,980,936,158.061188
254660,WagRES2,Wag,RES,2023-08-04 09:00:00,2023-08-04,9,216,31,8,3,...,9.752,795.887,736.133,738.408,781.432753,20,64,980,936,154.567247
254661,WagRES2,Wag,RES,2023-08-04 10:00:00,2023-08-04,10,216,31,8,3,...,10.531,796.181,736.337,738.612,782.656312,20,64,980,936,153.343688
254662,WagRES2,Wag,RES,2023-08-04 11:00:00,2023-08-04,11,216,31,8,3,...,11.410,NaN,736.517,738.792,NaN,20,64,980,936,NaN
